# Fine Tuning

## 数据分析

In [4]:
import pandas as pd

df = pd.read_csv('../data/texts/外卖评论/waimai_10k.csv')
df.head()


,label,review
0,1,很快，好吃，味道足，量大
1,1,没有送水没有送水没有送水
2,1,非常快，态度好。
3,1,方便，快捷，味道可口，快递给力
4,1,菜味道很棒！送餐很及时！


In [5]:
df.tail()

,label,review
11982,0,以前几乎天天吃，现在调料什么都不放，
11983,0,昨天订凉皮两份，什么调料都没有放，就放了点麻油，特别难吃，丢了一份，再也不想吃了
11984,0,"凉皮太辣,吃不下都"
11985,0,本来迟到了还自己点！！！
11986,0,肉夹馍不错，羊肉泡馍酱肉包很一般。凉面没想象中好吃。送餐倒是很快。


In [128]:
df_good = df[df.label == 1]
df_bad = df[df.label == 0]
print(f"外卖评论总数：{len(df)}，正面：{len(df_good)}，负面：{len(df_bad)}")

外卖评论总数：11987，正面：4000，负面：7987


## 制作数据集

### 获得子集

In [129]:
df_good_sample = df_good.sample(250)
df_bad_sample = df_bad.sample(250)
df_sample = pd.concat([df_good_sample, df_bad_sample])

In [153]:
# 数据打乱
df_sample = df_sample.sample(len(df_sample))

# 索引重置
df_sample = df_sample.reset_index(drop=True)

### 缓存 Embedding

In [214]:
import openai
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt


@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embeddings(texts: list[str], model="text-embedding-ada-002") -> list[list[float]]:
    response = openai.Embedding.create(input=texts, model=model)
    return [data["embedding"] for data in response["data"]]

# 采用批量 100 条评论的方式，获取评论的 Embedding
review_embeddings = []
for i in range(0, len(df_sample), 100):
    reviews = df_sample.review[i:i+100].to_list()
    review_embeddings.extend(get_embeddings(reviews))

df_sample["embedding"] = review_embeddings
df_sample.to_parquet('../data/texts/外卖评论/waimai_500.parquet')

### 加载本地数据集（包含 Embedding）

In [219]:
df_sample = pd.read_parquet('../data/texts/外卖评论/waimai_500.parquet')
df_sample.head()

,label,review,embedding
0,1,肘子卷饼肉很足。就是我口味重不够咸,"[0.004734907764941454, -0.018601905554533005, ..."
1,0,味道著實一般送到時候飯涼涼的,"[0.014507911168038845, -0.014204224571585655, ..."
2,1,味道都很好，就是终于饭点送餐慢,"[0.010158994235098362, -0.017008569091558456, ..."
3,1,物有所值，很给力,"[-0.02119990438222885, -0.026337601244449615, ..."
4,0,送达个屁啊！我这还没吃上呢，系统就显示已送达！！！我只能说这个软件是神马玩意啊！！！！,"[-0.009312949143350124, -0.022111525759100914,..."


In [220]:
# 分割训练集和测试集
df_train1, df_train2, df_test = df_sample.iloc[:300], df_sample.iloc[300:400], df_sample.iloc[400:]
print(f"训练集1：{len(df_train1)}，训练集2：{len(df_train2)}，测试集：{len(df_test)}")


训练集1：300，训练集2：100，测试集：100


## 查看使用 Embedding 计算的分类结果

In [173]:
# 余弦相似度计算
def get_score(embedding, class_embeddings):
  good_embedding, bad_embedding = class_embeddings
  return cosine_similarity(embedding, good_embedding) - cosine_similarity(embedding, bad_embedding)

In [175]:
good_bad_embeddings_en = get_embeddings(["good", "bad"])
good_bad_embeddings_zh = get_embeddings(["好", "坏"])

In [223]:
error_count = 0
for i, review_embedding in enumerate(review_embeddings):
    score = get_score(review_embedding, good_bad_embeddings_en)
    if not (score > 0 and df_sample.label[i] == 1 or score < 0 and df_sample.label[i] == 0):
        print(i, df_sample.review[i], score, df_sample.label[i])
        error_count += 1

print(f"错误数: {error_count}，错误率：{error_count / len(review_embeddings) * 100:.2f}%")

1 味道著實一般送到時候飯涼涼的 0.03305286968647281 0
4 送达个屁啊！我这还没吃上呢，系统就显示已送达！！！我只能说这个软件是神马玩意啊！！！！ 0.0160585019109013 0
7 量够，味道还行，包装不错 0.05374665368202858 0
11 小黄鱼贴饼子根本不是贴饼子，，炸窝头！都黑了！什么玩意！糊弄人呢！！！ -0.003980839681130721 1
12 晚了45分钟，套餐有半个卤蛋，但是没有饮料；,肥牛饭味道比较一般，但量很足（ps,配菜是豆芽菜，个人最不喜欢豆芽菜,赶上了……） 0.008583377161884842 0
16 菜也太少太少了吧，感觉跟别人吃剩了扒拉给我的一样 0.00965415256280533 0
17 这馅料的量，我只有呵呵了 0.016350143485567292 0
18 送的太慢了1个半小时 0.0001338335638244592 0
20 分量很小,,拌饭还是凉的 0.008598353013520632 0
24 挺好哒。米饭很足。很容易饱。 0.04877359574426765 0
25 饼粘糊糊的,里面的还行 0.023990130610683003 0
36 送的倒是挺快的，只是汉堡做那么小好意思吗？ 0.02059649931321017 0
42 鸡肉不好吃，多弄点肘子，不多说 -0.0015058422395071158 1
56 星巴克永远洒半杯，下次收半价行嘛？ 0.0030364002978980897 0
59 就薯条还行！其他都不怎么样！ 0.03051034327741764 0
60 外卖餐盒还要钱不合理，而且餐盒很贵，几分钱的餐盒要一块一个 0.0039596649088344815 0
68 晕了都，让12点送过来，你这10.50多就送过来了，让我情何以堪 0.014872632326197843 0
76 我要的是冰咖啡，给我送来热的 0.020295307277199526 0
84 泡菜炒饭，少了芝士啊！ 0.00018283260830653791 0
95 大过年的啥也不说了，18点10分下单，20点20送到家，1公里多的路，菜全都冰凉，羊棒骨不给吸管怎么吃？以后再也不叫他家外卖！！ 0.0080929323886804

## 微调模型

### 准备数据集

In [230]:
texts = df_sample.review.to_list()
labels = df_sample.label.apply(lambda x: "good" if x == 1 else "bad").to_list()

df = pd.DataFrame({"prompt": texts, "completion": labels})
df.head()

,prompt,completion
0,肘子卷饼肉很足。就是我口味重不够咸,good
1,味道著實一般送到時候飯涼涼的,bad
2,味道都很好，就是终于饭点送餐慢,good
3,物有所值，很给力,good
4,送达个屁啊！我这还没吃上呢，系统就显示已送达！！！我只能说这个软件是神马玩意啊！！！！,bad


In [234]:
df.to_json("../data/texts/外卖评论/waimai_500.jsonl", orient='records', lines=True)


In [235]:
!openai tools fine_tunes.prepare_data -f ../data/texts/外卖评论/waimai_500.jsonl -q

Analyzing...

- Your file contains 500 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- More than a third of your `prompt` column/key is uppercase. Uppercase prompts tends to perform worse than a mixture of case encountered in normal language. We recommend to lower case the data if that makes sense in your domain. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for 

### 手动整理出了 5 个jsonl 文件。
* ../data/texts/外卖评论/waimai_500_prepared_1_train.jsonl  第一次训练的数据集
* ../data/texts/外卖评论/waimai_500_prepared_1_valid.jsonl  第一次验证的数据集
* ../data/texts/外卖评论/waimai_500_prepared_2_train.jsonl  第二次训练的增量数据集
* ../data/texts/外卖评论/waimai_500_prepared_2_valid.jsonl  第二次验证的增量数据集
* ../data/texts/外卖评论/waimai_500_prepared_test.jsonl     测试数据集

### 创建微调模型

In [ ]:
# For binary classification
!openai api fine_tunes.create \
  -t ../data/texts/外卖评论/waimai_500_prepared_1_train.jsonl \
  -v ../data/texts/外卖评论/waimai_500_prepared_1_valid.jsonl \
  -m ada \
  --compute_classification_metrics \
  --classification_n_classes 2 \
  --classification_positive_class " good"